In [2]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [3]:
AGENT_NAME = "at020-state-machine"

In [4]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.

from typing import cast
import uuid

from agentex.types import Task

TASK_ID = str(uuid.uuid4())[:8]

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="task/create",
    params={
        "name": f"{TASK_ID}-task",
        "params": {}
    }
)

task = cast(Task, rpc_response.result)
print(task)

Task(id='09121bce-e4d6-452d-82be-7bb9de75a24e', created_at=datetime.datetime(2025, 7, 27, 6, 3, 27, 74360, tzinfo=TzInfo(UTC)), name='2191ef3a-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 27, 6, 3, 27, 74360, tzinfo=TzInfo(UTC)))


In [5]:
# Test non streaming response
from typing import cast
from agentex.types import Event

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="event/send",
    params={
        "content": {"type": "text", "author": "user", "content": "Hello tell me the latest news about AI and AI startups"},
        "task_id": task.id,
    }
)

event = cast(Event, rpc_response.result)
print(event)

Event(id='b379c00e-b30a-4808-b0fe-4c6ffa6acd5b', agent_id='8e8d7d03-214f-4163-97b4-ce687ce9923f', sequence_id=231, task_id='09121bce-e4d6-452d-82be-7bb9de75a24e', content=TextContent(author='user', content='Hello tell me the latest news about AI and AI startups', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 27, 6, 3, 27, 572883, tzinfo=TzInfo(UTC)))


In [7]:
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=20,
)

╭───────────────────────── USER [07/27/2025 06:03:27] ─────────────────────────╮
│ Hello tell me the latest news about AI and AI startups                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/27/2025 06:03:31] ─────────────────────────╮
│ 🔧 Tool Request: web_search                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "input": "latest news about AI and AI startups",                          │
│    "user_location": {                                                        │
│      "type": "approximate",                                                  │
│      "city": "New York",                                                     │
│      "country": "USA",                                                       │
│      "region": "NY",                                                         │
│      "timezone": "America/New_York"                                          │
│    }                                                                         │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/27/2025 06:03:31] ─────────────────────────╮
│ ✅ Tool Response: web_search                                                 │
│                                                                              │
│ Response:                                                                    │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "type": "text",                                                           │
│    "text": "Error executing tool web_search: Error code: 400 - {'error':     │
│  {'message': \"Invalid input USA: 'country' must be an ISO 3166-1 code       │
│  (https://en.wikipedia.org/wiki/ISO_3166-1).\", 'type':                      │
│  'invalid_request_error', 'param': 'tools', 'code': None}}",                 │
│    "annotations": null,                                                      │
│    "meta": null                                                              │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/27/2025 06:03:39] ─────────────────────────╮
│ 🔧 Tool Request: web_search                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "input": "latest news about AI and AI startups",                          │
│    "user_location": {                                                        │
│      "type": "approximate",                                                  │
│      "city": "New York",                                                     │
│      "country": "US",                                                        │
│      "region": "NY",                                                         │
│      "timezone": "America/New_York"                                          │
│    }                                                                         │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/27/2025 06:03:39] ─────────────────────────╮
│ ✅ Tool Response: web_search                                                 │
│                                                                              │
│ Response:                                                                    │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "type": "text",                                                           │
│    "text": "Artificial intelligence (AI) continues to be a dynamic sector,   │
│  with numerous startups making significant strides in technology developmen  │
│  and securing substantial funding. Here's an overview of recent              │
│  developments:\n\n**Major Funding Rounds:**\n\n- **Perplexity AI:** This AI  │
│  search startup has achieved a valuation of $18 billion following a $100     │
│  million funding round, reflecting strong investor interest in the           │
│  competitive AI search market.                                               │
│  ([pymnts.com](https://www.pymnts.com/news/artificial-intelligence/2025/thi  │
│  week-in-ai-ai-startups-hit-fundraising-gold/?utm_source=openai))\n\n-       │
│  **Thinking Machines:** Founded by former OpenAI CTO Mira Murati, this       │
│  startup raised $2 billion, valuing the company at $10 billion.              │
│  ([pymnts.com](https://www.pymnts.com/news/artificial-intelligence/2025/thi  │
│  week-in-ai-ai-startups-hit-fundraising-gold/?utm_source=openai))\n\n-       │
│  **Harvey:** Specializing in AI-driven legal solutions, Harvey secured $300  │
│  million in a Series D funding round, bringing its valuation to $3 billion.  │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Harvey_%28software%29?ut  │
│  source=openai))\n\n**Notable Acquisitions:**\n\n- **Amazon's Acquisition o  │
│  Bee:** Amazon announced plans to acquire Bee, a San Francisco-based startu  │
│  developing AI-powered wearable technology. Bee's flagship product is a $50  │
│  AI-enabled wristband capable of transcribing conversations and generating   │
│  summaries or to-do lists from the recordings.                               │
│  ([reuters.com](https://www.reuters.com/business/retail-consumer/amazon-buy  │
│  tartup-focused-ai-wearables-2025-07-22/?utm_source=openai))\n\n**Industry   │
│  Trends:**\n\n- **AI Dominates Venture Capital Funding:** In the first half  │
│  of 2025, AI startups accounted for 53% of global venture capital            │
│  investments, with 64% in the U.S., indicating a transformative period for   │
│  tech investment.                                                            │
│  ([axios.com](https://www.axios.com/newsletters/axios-pro-rata-d4299627-1e8  │
│  44f2-9308-212d8860b6aa?utm_source=openai))\n\n- **China's AI Cooperation    │
│  Initiative:** China proposed the creation of a new international            │
│  organization to promote global cooperation on AI, aiming to provide an      │
│  alternative to U.S.-led initiatives and foster inclusive development of th  │
│  technology.                                                                 │
│  ([reuters.com](https://www.reuters.com/world/china/china-proposes-new-glob  │
│  -ai-cooperation-organisation-2025-07-26/?utm_source=openai))\n\n**Emerging  │
│  Startups:**\n\n- **Mistral AI:** Founded in 2023, Mistral AI specializes i  │
│  open-weight large language models and has secured significant funding,      │
│  including a \u20ac600 million round in June 2024, elevating its valuation   │
│  \u20ac5.8 billion.                                                          │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Mistral_AI?utm_source=op  │
│  ai))\n\n- **Neysa:** Established in 2023, Neysa provides a cloud platform   │
│  for AI acceleration and high-

╭──────────────────────── AGENT [07/27/2025 06:03:40] ─────────────────────────╮
│ Here's the latest news about AI and AI startups:                             │
│                                                                              │
│                            Major Funding Rounds:                             │
│                                                                              │
│  • Perplexity AI: This AI search startup has secured a $100 million funding  │
│    round, bringing its valuation to $18 billion. This reflects the strong    │
│    interest in AI search technology.                                         │
│  • Thinking Machines: Founded by former OpenAI CTO Mira Murati, this startup │
│    raised $2 billion, with a valuation reaching $10 billion.                 │
│  • Harvey: Specializing in AI-driven legal solutions, Harvey raised $300     │
│    million in a Series D round, resulting in a valuation of $3 billion.      │
│                                                                              │
│                            Notable Acquisitions:                             │
│                                                                              │
│  • Amazon's Acquisition of Bee: Amazon plans to acquire Bee, a San           │
│    Francisco-based startup focused on AI-powered wearables. Bee’s flagship   │
│    product is an AI-enabled wristband priced at $50 that can transcribe      │
│    conversations and generate summaries or to-do lists.                      │
│                                                                              │
│                               Industry Trends:                               │
│                                                                              │
│  • AI Venture Capital Dominance: In the first half of 2025, AI startups      │
│    accounted for 53% of global venture capital investments, with a           │
│    significant portion (64%) in the U.S.                                     │
│  • China's AI Cooperation Initiative: China has proposed creating a new      │
│    international organization to promote global AI cooperation, aiming to    │
│    foster inclusive development and provide an alternative to U.S.-led       │
│    initiatives.                                                              │
│                                                                              │
│                              Emerging Startups:                              │
│                                                                              │
│  • Mistral AI: Specializing in open-weight large language models, Mistral AI │
│    has secured significant funding, including €600 million in June 2024,     │
│    with a valuation now at €5.8 billion.                                     │
│  • Neysa: Offering a cloud platform for AI acceleration, Neysa has raised a  │
│    total of $50 million across two major funding rounds.                     │
│                                                                              │
│ For more details, you may visit their respective announcements on sites like │
│ Reuters, Wikipedia, and Axios.                                               │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 20 seconds - returning collected messages
